In [1]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, Aer, transpile, assemble, circuit
from qiskit.providers.aer import QasmSimulator
from qiskit.visualization import plot_histogram
import numpy as np

In [2]:
def gcd(a,b):
    if a < b:
        return gcd(b,a)
    if b == 0:
        return a
    return gcd(a%b, b)

In [3]:
def mod_by_rep_square(num, repetitions, modulus):
    for i in range(repetitions):
        num = np.mod(num**2, modulus)
    return num

In [4]:
def exponent_to_factors(guess, exponent, num):
    if r % 2 == 1:
        return (1, num)
    else:
        return (gcd(guess ** exponent + 1, num), gcd(guess**exponent - 1, num))

In [5]:
def classical_add(bits, add):
    """ Creates a Quantum Circuit that increments a register of qubits in the Fourier Space by the parameter add """
    c_add = QuantumCircuit(bits)
    bin_add = bin(add)[2:]
    while len(bin_add) < bits:
        bin_add = '0' + bin_add
    for cbit in range(len(bin_add)):
        if bin_add[cbit] == '1':
            for qbit in range(bits):
                c_add.p( (np.pi/(2**(cbit - qbit))), bits - qbit - 1)
    return c_add
def classical_add_gate(bits, add):
    c_add = classical_add(bits, add).to_instruction()
    c_add.name = "add {}".format(add)
    return c_add
def controlled_classical_add_gate(bits, add):
    c_add = classical_add(bits, add)
    c_add.name = "add {}".format(add)
    return c_add.control(num_ctrl_qubits=1).to_instruction()

In [6]:
def quantum_add(n1, n2, bits):
    """ Use QFT, classical_add, and Inverse QFT to add two numbers """
    if n1 > (2**(bits - 2)) or n2 > (2**(bits - 2)):
        raise OverflowError
    bn1 = bin(n1)[2:]
    while len(bn1) < bits:
        bn1 = '0' + bn1
    qr = QuantumRegister(bits, "q")
    qc = QuantumCircuit(qr)
    for bit in range(len(bn1)):
        if bn1[bit] == '1':
            qc.x(qr[bits - bit - 1])
    where = []
    for i in range(len(qr)):
        where.append(qr[i])
    qc.append(circuit.library.QFT(num_qubits = bits, do_swaps = False).to_instruction(), where)
    qc.append(classical_add_gate(bits, n2), where)
    qc.append(circuit.library.QFT(num_qubits = bits, do_swaps = False).inverse().to_instruction(), where)
    qc.measure_all()
    return qc

In [7]:
q = quantum_add(1, 4, 4)
q.draw()

┌───┐┌──────┐┌────────┐┌───────┐ ░ ┌─┐         
   q_0: ┤ X ├┤0     ├┤0       ├┤0      ├─░─┤M├─────────
        └───┘│      ││        ││       │ ░ └╥┘┌─┐      
   q_1: ─────┤1     ├┤1       ├┤1      ├─░──╫─┤M├──────
             │  qft ││  add 4 ││  iqft │ ░  ║ └╥┘┌─┐   
   q_2: ─────┤2     ├┤2       ├┤2      ├─░──╫──╫─┤M├───
             │      ││        ││       │ ░  ║  ║ └╥┘┌─┐
   q_3: ─────┤3     ├┤3       ├┤3      ├─░──╫──╫──╫─┤M├
             └──────┘└────────┘└───────┘ ░  ║  ║  ║ └╥┘
meas: 4/════════════════════════════════════╩══╩══╩══╩═
                                            0  1  2  3

In [8]:
backend = QasmSimulator()
q_comp = transpile(q, backend)
job = backend.run(q_comp, shots = 10)
result = job.result()
result.get_counts(q_comp)

{'0101': 10}

In [20]:
def doubly_controlled_modular_add(b, m, bits):
    """ Use the Quantum_Add Gate to add to a number in the modular base """
    qr = QuantumRegister(bits, "q")
    ar = QuantumRegister(1, "a")
    qc = QuantumCircuit(qr, ar)
    where = []
    for i in range(bits):
        where.append(qr[i])
    qc.append(classical_add_gate(bits, b), where)
    qc.append(classical_add_gate(bits, m).inverse(), where)
    qc.append(circuit.library.QFT(num_qubits=bits, do_swaps=False).inverse().to_instruction(), where)
    qc.cx(qr[bits-1],ar[0])
    qc.append(circuit.library.QFT(num_qubits=bits, do_swaps=False).to_instruction(), where)
    where2 = [ar[0]] + where
    qc.append(controlled_classical_add_gate(bits, m), where2)
    qc.append(classical_add_gate(bits, b).inverse(), where)
    qc.append(circuit.library.QFT(num_qubits=bits, do_swaps=False).inverse().to_instruction(), where)
    qc.x(qr[bits-1])
    qc.cx(qr[bits-1],ar[0])
    qc.x(qr[bits-1])
    qc.append(circuit.library.QFT(num_qubits=bits, do_swaps=False).to_instruction(), where)
    qc.append(classical_add_gate(bits, b), where)
    return qc

In [21]:
q = doubly_controlled_modular_add(4, 10, 5)
q.draw()

┌────────┐┌────────────┐┌───────┐     ┌──────┐┌───────────┐┌───────────┐»
q_0: ┤0       ├┤0           ├┤0      ├─────┤0     ├┤1          ├┤0          ├»
     │        ││            ││       │     │      ││           ││           │»
q_1: ┤1       ├┤1           ├┤1      ├─────┤1     ├┤2          ├┤1          ├»
     │        ││            ││       │     │      ││           ││           │»
q_2: ┤2 add 4 ├┤2 add 10_dg ├┤2 iqft ├─────┤2 qft ├┤3          ├┤2 add 4_dg ├»
     │        ││            ││       │     │      ││  c_add 10 ││           │»
q_3: ┤3       ├┤3           ├┤3      ├─────┤3     ├┤4          ├┤3          ├»
     │        ││            ││       │     │      ││           ││           │»
q_4: ┤4       ├┤4           ├┤4      ├──■──┤4     ├┤5          ├┤4          ├»
     └────────┘└────────────┘└───────┘┌─┴─┐└──────┘│           │└───────────┘»
a_0: ─────────────────────────────────┤ X ├────────┤0          ├─────────────»
                                      └───┘        └───────────┘             »
«     ┌───────┐               ┌──────┐┌────────┐
«q_0: ┤0      ├───────────────┤0     ├┤0       ├
«     │       │               │      ││        │
«q_1: ┤1      ├───────────────┤1     ├┤1       ├
«     │       │               │      ││        │
«q_2: ┤2 iqft ├───────────────┤2 qft ├┤2 add 4 ├
«     │       │               │      ││        │
«q_3: ┤3      ├───────────────┤3     ├┤3       ├
«     │       │┌───┐     ┌───┐│      ││        │
«q_4: ┤4      ├┤ X ├──■──┤ X ├┤4     ├┤4       ├
«     └───────┘└───┘┌─┴─┐└───┘└──────┘└────────┘
«a_0: ──────────────┤ X ├───────────────────────
«                   └───┘

In [29]:
def test_dcma_gate(a, b, m, bits):
    bina = bin(a)[2:]
    while(len(bina) < bits):
        bina = '0' + bina
    qr = QuantumRegister(bits, "q")
    ar = QuantumRegister(1, "a")
    qc = QuantumCircuit(qr, ar)
    for bit in range(len(bina)):
        if bina[bit] == '1':
            qc.x(qr[bits - bit - 1])
    where = []
    for i in range(len(qr)):
        where.append(qr[i])
    qc.append(circuit.library.QFT(num_qubits=bits, do_swaps=False).to_instruction(), where)
    where2 = where + [ar[0]]
    qc.append(doubly_controlled_modular_add(b, m, bits).to_instruction(), where2)
    qc.append(circuit.library.QFT(num_qubits=bits, do_swaps=False).inverse().to_instruction(), where)
    qc.measure_all()
    return qc
q = test_dcma_gate(3,4,6,6)
q.draw()
    

┌───┐┌──────┐┌──────────────┐┌───────┐ ░ ┌─┐                  
   q_0: ┤ X ├┤0     ├┤0             ├┤0      ├─░─┤M├──────────────────
        ├───┤│      ││              ││       │ ░ └╥┘┌─┐               
   q_1: ┤ X ├┤1     ├┤1             ├┤1      ├─░──╫─┤M├───────────────
        └───┘│      ││              ││       │ ░  ║ └╥┘┌─┐            
   q_2: ─────┤2     ├┤2             ├┤2      ├─░──╫──╫─┤M├────────────
             │  qft ││              ││  iqft │ ░  ║  ║ └╥┘┌─┐         
   q_3: ─────┤3     ├┤3 circuit-699 ├┤3      ├─░──╫──╫──╫─┤M├─────────
             │      ││              ││       │ ░  ║  ║  ║ └╥┘┌─┐      
   q_4: ─────┤4     ├┤4             ├┤4      ├─░──╫──╫──╫──╫─┤M├──────
             │      ││              ││       │ ░  ║  ║  ║  ║ └╥┘┌─┐   
   q_5: ─────┤5     ├┤5             ├┤5      ├─░──╫──╫──╫──╫──╫─┤M├───
             └──────┘│              │└───────┘ ░  ║  ║  ║  ║  ║ └╥┘┌─┐
   a_0: ─────────────┤6             ├──────────░──╫──╫──╫──╫──╫──╫─┤M├
                     └──────────────┘          ░  ║  ║  ║  ║  ║  ║ └╥┘
meas: 7/══════════════════════════════════════════╩══╩══╩══╩══╩══╩══╩═
                                                  0  1  2  3  4  5  6

In [30]:
q_comp = transpile(q, backend)
job = backend.run(q_comp, shots = 10)
result = job.result()
result.get_counts(q_comp)

{'0000001': 10}

In [31]:
#a = q.decompose()
#a.draw()